
# 🚀 Guia Completo do SQLAlchemy ORM

Neste notebook, você aprenderá tudo sobre **SQLAlchemy ORM**, desde os conceitos fundamentais até exemplos práticos avançados.  
Vamos explorar como **manipular bancos de dados** sem precisar escrever SQL puro, utilizando a técnica de **Object-Relational Mapping (ORM)**.

---

## 📌 **O que é ORM e por que usar?**

ORM (**Object-Relational Mapping**) permite manipular bancos de dados usando **classes e objetos** em Python, ao invés de escrever **SQL manualmente**.  
Isso traz várias vantagens:

✔ **Facilidade de uso** → Trabalhamos com classes Python em vez de consultas SQL complexas.  
✔ **Portabilidade** → Podemos trocar o banco de dados (SQLite, MySQL, PostgreSQL, etc.) sem modificar o código.  
✔ **Segurança** → Evita SQL Injection, pois as consultas são geradas automaticamente pelo SQLAlchemy.  
✔ **Reutilização de código** → Classes podem ser reutilizadas e ampliadas em diferentes partes do programa.  

O **SQLAlchemy** é a biblioteca ORM mais popular do Python e permite transformar tabelas do banco de dados em **classes Python**, facilitando a manipulação dos dados.

---

## 📌 **Configuração do SQLAlchemy**

Vamos configurar um **banco de dados SQLite** e criar uma **classe ORM** que representa a tabela `pedidos`.  

---


## **Criando a Conexão com o Banco**

O primeiro passo no SQLAlchemy é criar a conexão com o banco de dados.

In [1]:
from sqlalchemy import create_engine

# Criando conexão com o banco SQLite
engine = create_engine("sqlite:///banco_exemplo.db", echo=True)

/// ou ////

**Resumo**
- `create_engine()` cria a conexão com o banco.
- `"sqlite:///meu_banco.db"` indica que estamos usando um banco SQLite chamado `meu_banco.db`.
- `echo=True` ativa logs para visualizar os comandos SQL gerados.

## **Criando uma Classe Base**

O SQLAlchemy usa classes Python para representar tabelas do banco.  
Para isso, precisamos de uma **classe base** para criar nossas tabelas.

In [2]:
from sqlalchemy.orm import declarative_base

# Criando a base para o ORM
Base = declarative_base()

**Resumo**
- `declarative_base()` cria uma classe base para todas as tabelas do banco.
- Todas as classes que criarmos herdarão dessa base

## **Definindo uma Tabela (Criando uma Classe ORM)**

Agora, vamos criar uma classe Python que representa uma tabela no banco.

In [3]:
from sqlalchemy import Column, Integer, String

class Usuario(Base):
    __tablename__ = "usuarios"  # Nome da tabela no banco

    id = Column(Integer, primary_key=True)  # Chave primária
    nome = Column(String, nullable=False)  # Nome do usuário
    idade = Column(Integer, nullable=False)  # Idade do usuário

    def __repr__(self):
        return f'{self.nome}, {self.idade}'

**Resumo**
- Criamos a classe `Usuario`, que representa uma tabela chamada `"usuarios"`.
- `id` → Chave primária da tabela.
- `nome` → Coluna do tipo `String`, que não pode ser `NULL`.
- `idade` → Coluna do tipo `Integer`, que também não pode ser `NULL`.

## **Criando as Tabelas no Banco**

Agora que definimos a classe `Usuario`, precisamos **criar essa tabela no banco**.

In [4]:
# Criando a tabela no banco de dados
Base.metadata.create_all(engine)

2025-11-02 13:02:35,684 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-02 13:02:35,685 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("usuarios")
2025-11-02 13:02:35,685 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-02 13:02:35,686 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("usuarios")
2025-11-02 13:02:35,686 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-11-02 13:02:35,687 INFO sqlalchemy.engine.Engine 
CREATE TABLE usuarios (
	id INTEGER NOT NULL, 
	nome VARCHAR NOT NULL, 
	idade INTEGER NOT NULL, 
	PRIMARY KEY (id)
)


2025-11-02 13:02:35,687 INFO sqlalchemy.engine.Engine [no key 0.00030s] ()
2025-11-02 13:02:35,696 INFO sqlalchemy.engine.Engine COMMIT


**Resumo**
- O SQLAlchemy **verifica se a tabela já existe** no banco.
- Se **não existir**, ele **cria a tabela automaticamente**.

## **Criando uma Sessão para Manipular o Banco**

Para adicionar, buscar ou atualizar registros, precisamos de uma **sessão**.

In [5]:
from sqlalchemy.orm import sessionmaker

# Criando uma fábrica de sessões
Session = sessionmaker(bind=engine)

# Criando uma sessão
session = Session()

**Resumo**
- `sessionmaker(bind=engine)` cria uma **fábrica de sessões** conectada ao banco.
- `Session()` inicia uma **nova sessão**, permitindo executar operações no banco.

## **Adicionando Registros ao Banco**

In [6]:
# Criando um novo usuário
novo_usuario = Usuario(nome="Carlos", idade=30)

# Adicionando à sessão
session.add(novo_usuario)

# Salvando no banco
session.commit()

2025-11-02 13:02:38,029 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-02 13:02:38,033 INFO sqlalchemy.engine.Engine INSERT INTO usuarios (nome, idade) VALUES (?, ?)
2025-11-02 13:02:38,034 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ('Carlos', 30)
2025-11-02 13:02:38,036 INFO sqlalchemy.engine.Engine COMMIT


**Resumo**
- Criamos um objeto `Usuario` (representando um registro na tabela).
- `session.add(novo_usuario)` adiciona o objeto na sessão.
- `session.commit()` grava os dados no banco.

## **Consultando Registros no Banco**

Agora, vamos buscar um usuário pelo nome.

In [7]:
# Buscando um usuário com nome "Fernando"
usuario = session.query(Usuario).filter_by(nome="Carlos").first()
print('Nome:', usuario.nome, ', Idade:', usuario.idade)

2025-11-02 13:02:38,984 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-02 13:02:38,990 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nome AS usuarios_nome, usuarios.idade AS usuarios_idade 
FROM usuarios 
WHERE usuarios.nome = ?
 LIMIT ? OFFSET ?
2025-11-02 13:02:38,991 INFO sqlalchemy.engine.Engine [generated in 0.00128s] ('Carlos', 1, 0)
Nome: Carlos , Idade: 30


In [8]:
# print(usuario)

**Resumo**
- `session.query(Usuario)` → Inicia uma consulta na tabela `usuarios`.
- `filter_by(nome="Carlos")` → Filtra apenas os usuários com `nome = "Carlos"`.
- `first()` → Retorna apenas o primeiro resultado encontrado.

## **Atualizando um Registro no Banco**

In [9]:
# Alterando a idade do usuário "Fernando"
usuario.idade = 35

# Salvando a alteração no banco
session.commit()

2025-11-02 13:02:40,280 INFO sqlalchemy.engine.Engine UPDATE usuarios SET idade=? WHERE usuarios.id = ?
2025-11-02 13:02:40,281 INFO sqlalchemy.engine.Engine [generated in 0.00071s] (35, 1)
2025-11-02 13:02:40,282 INFO sqlalchemy.engine.Engine COMMIT


**Resumo**
- Alteramos o valor do atributo `idade` do objeto `usuario`.
- `session.commit()` grava a alteração no banco de dados.

## **Removendo um Registro do Banco**

In [10]:
# Deletando o usuário "Fernando"
session.delete(usuario)

# Confirmando a exclusão
session.commit()

2025-11-02 13:02:41,317 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-02 13:02:41,319 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nome AS usuarios_nome, usuarios.idade AS usuarios_idade 
FROM usuarios 
WHERE usuarios.id = ?
2025-11-02 13:02:41,319 INFO sqlalchemy.engine.Engine [generated in 0.00032s] (1,)
2025-11-02 13:02:41,320 INFO sqlalchemy.engine.Engine DELETE FROM usuarios WHERE usuarios.id = ?
2025-11-02 13:02:41,320 INFO sqlalchemy.engine.Engine [generated in 0.00024s] (1,)
2025-11-02 13:02:41,321 INFO sqlalchemy.engine.Engine COMMIT


In [11]:
session.query(Usuario).filter_by(nome="Carlos").first()

2025-11-02 13:02:41,590 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-02 13:02:41,592 INFO sqlalchemy.engine.Engine SELECT usuarios.id AS usuarios_id, usuarios.nome AS usuarios_nome, usuarios.idade AS usuarios_idade 
FROM usuarios 
WHERE usuarios.nome = ?
 LIMIT ? OFFSET ?
2025-11-02 13:02:41,593 INFO sqlalchemy.engine.Engine [cached since 2.603s ago] ('Carlos', 1, 0)


**Resumo**
- `session.delete(usuario)` marca o objeto para exclusão.
- `session.commit()` confirma a remoção no banco.


# 📌 **Resumo das Operações SQLAlchemy**

| Passo | Comando | Explicação |
|-------|---------|------------|
| **1️⃣ Criar conexão** | `engine = create_engine("sqlite:///meu_banco.db")` | Conexão com o banco |
| **2️⃣ Criar base ORM** | `Base = declarative_base()` | Define a estrutura ORM |
| **3️⃣ Criar tabela** | Criar classe herdando de `Base` | Define a estrutura da tabela |
| **4️⃣ Criar tabelas no banco** | `Base.metadata.create_all(engine)` | Cria a tabela se não existir |
| **5️⃣ Criar sessão** | `session = Session()` | Inicia a sessão para interagir com o banco |
| **6️⃣ Adicionar registro** | `session.add(objeto)` + `session.commit()` | Insere um novo dado |
| **7️⃣ Buscar dados** | `session.query(Tabela).filter_by(coluna=valor).first()` | Consulta dados no banco |
| **8️⃣ Atualizar dados** | `objeto.atributo = novo_valor` + `session.commit()` | Atualiza um valor |
| **9️⃣ Deletar dados** | `session.delete(objeto)` + `session.commit()` | Remove um dado |

# SQLAlchemy ORM — Passo a passo (Resumo, com ajustes)

> Fluxo mental: **Engine → Base → Modelos → Criar Tabelas → Sessão → CRUD**

## 1) Engine (conexão com o banco)
- Crie o *engine*, apontando para o banco (ex.: arquivo SQLite).
- O parâmetro `echo` liga/desliga os logs SQL (útil para depuração).

## 2) Base declarativa
- Importe/crie a **Base** declarativa.
- Todas as classes de modelo (tabelas) **herdam dessa Base**.

## 3) Modelo (classe ORM = tabela)
- Crie uma classe por tabela, **herdando de Base**.
- Defina `__tablename__` e as **colunas**.
- Garanta **uma chave primária**.

## 4) Criar as tabelas no banco
- Use o *metadata* da Base para **materializar** as tabelas no banco: `create_all` (com underscore).
- Observação: `create_all` **não recria/apaga** automaticamente tabelas existentes; cria somente se não existirem.

## 5) Sessão (onde o CRUD acontece)
- **Opção clássica:** crie uma **fábrica de sessões** com `sessionmaker` (minúsculo) vinculada ao *engine* e então a **sessão**.
- **Estilo 2.0 (recomendado):** use `with Session(engine) as session:` e, de preferência, `with session.begin():` para garantir **commit/rollback** automáticos.
- **Mudanças** (insert/update/delete) exigem **commit**; **leituras** não.

## 6) CRUD — visão rápida
- **Create**: instancie o modelo, **adicione à sessão** e **commit**.
- **Read (duas formas):**
  - *Clássica*: `session.query(Model).filter_by(...).first()/all()`.
  - *2.0 moderna*: `session.execute(select(Model).where(...)).scalars().first()/all()`.
- **Update**: recupere a instância, **altere atributos** e **commit**.
- **Delete**: **delete a instância** e **commit**.

## 7) Boas práticas
- Comece com **SQLite**; para Postgres/MySQL, ajuste a **string de conexão** e instale o **driver** adequado (ex.: `psycopg` para Postgres).
- Padronize nomes (ex.: `snake_case`) e avalie `unique`, `nullable`, `index` nas colunas.
- Prefira **context manager**: `with Session(engine) as session, session.begin(): ...` para transações seguras.
